In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import kurtosis
from scipy.stats import skew
import seaborn as sns
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV,KFold

In [3]:
train1 = pd.read_csv(r"F:\project\train.csv",dtype={'acoustic_data':np.int16, 'time_to_failure':np.float64},nrows=153600000)

In [4]:
train2 = pd.read_csv(r"F:\project\train.csv",dtype={'acoustic_data':np.int16, 'time_to_failure':np.float64},skiprows=153600000,nrows=153600000,names=['acoustic_data','time_to_failure'])
#train2.columns['acoustic_data','time_to_failure']


In [5]:
train3 = pd.read_csv(r"F:\project\train.csv",dtype={'acoustic_data':np.int16, 'time_to_failure':np.float64},skiprows=153600000*2,nrows=153600000,names=['acoustic_data','time_to_failure'])

In [6]:
train4 = pd.read_csv(r"F:\project\train.csv",dtype={'acoustic_data':np.int16, 'time_to_failure':np.float64},skiprows=153600000*3,nrows=153600000,names=['acoustic_data','time_to_failure'])

In [17]:
rows = 150000
segments = int(np.floor(train1.shape[0] / rows))
print("Number of segments: ", segments)
print(type(segments))

X_train = pd.DataFrame(index = range(segments), dtype = np.float64, columns = ['ave', 'std', 'max', 'min','sum','skew','kurt','var','quantile_1','quantile_2','quantile_3','median'])
y_train = pd.DataFrame(index = range(segments), dtype= np.float64, columns = ['time_to_failure'])

Number of segments:  1024
<class 'int'>


In [20]:
def add_features(data,n):
    for i in range(segments):
        # Selecting the rows from each segment in the data.
        seg = data.iloc[i*rows : (i*rows)+rows]
        i = i+1024*n
        
        x = seg['acoustic_data'].values
        y = seg['time_to_failure'].values
        #y = pd.DataFrame(y)
    
        y_train.loc[i,'time_to_failure'] = y.mean()
        #print(y_train.head())
        
        X_train.loc[i, 'ave'] = x.mean()
        X_train.loc[i, 'std'] = x.std()
        X_train.loc[i, 'max'] = x.max()
        X_train.loc[i, 'min'] = x.min()
        X_train.loc[i, 'sum'] = x.sum()
        X_train.loc[i, 'skew'] = skew(x)
        X_train.loc[i, 'kurt'] = kurtosis(x)
        X_train.loc[i,'var'] = x.var()
        X_train.loc[i,'quantile_1'] = seg['acoustic_data'].quantile(0.25)
        X_train.loc[i,'quantile_2'] = seg['acoustic_data'].quantile(0.50)
        X_train.loc[i,'quantile_3'] = seg['acoustic_data'].quantile(0.75)
        X_train.loc[i,'median'] = seg['acoustic_data'].median()
        #print(X_train.head())

#add_features(train1,0)
#print(X_train.shape)
#print(y_train.shape)

In [21]:
add_features(train1,0)

In [22]:
print(X_train.shape)
print(y_train.shape)

(1024, 12)
(1024, 1)


In [28]:
print(X_train.isna().sum().any())

False


In [29]:
add_features(train2,1)

In [30]:
add_features(train3,2)

In [31]:
add_features(train4,3)

In [32]:
print(X_train.shape)
print(y_train.shape)

(4096, 12)
(4096, 1)


In [33]:
X_train.isna().sum().any()

False

In [34]:
submission = pd.read_csv(r"D:\Project Dataset\LANL-Earthquake-Prediction\sample_submission.csv", index_col='seg_id')
submission.head()

,time_to_failure
seg_id,
seg_00030f,0
seg_0012b5,0
seg_00184e,0
seg_003339,0
seg_0042cc,0


In [35]:
X_test = pd.DataFrame(columns=X_train.columns, dtype=np.float64, index=submission.index)

In [36]:
for seg_id in X_test.index:
    seg = pd.read_csv("D:/Project Dataset/LANL-Earthquake-Prediction/test/" + seg_id + ".csv")
    
    x = seg['acoustic_data'].values
    
    X_test.loc[seg_id, 'ave'] = x.mean()
    X_test.loc[seg_id, 'std'] = x.std()
    X_test.loc[seg_id, 'max'] = x.max()
    X_test.loc[seg_id, 'min'] = x.min()
    X_test.loc[seg_id, 'sum'] = x.sum()
    X_test.loc[seg_id, 'skew'] = skew(x)
    X_test.loc[seg_id, 'kurt'] = kurtosis(x)
    X_test.loc[seg_id,'var'] = x.var()
    X_test.loc[seg_id,'quantile_1'] = seg['acoustic_data'].quantile(0.25)
    X_test.loc[seg_id,'quantile_2'] = seg['acoustic_data'].quantile(0.50)
    X_test.loc[seg_id,'quantile_3'] = seg['acoustic_data'].quantile(0.75)
    X_test.loc[seg_id,'median'] = seg['acoustic_data'].median()

In [38]:
print(X_test)
X_test.isna().sum().any()

                 ave        std    max    min       sum      skew        kurt  \
seg_id                                                                          
seg_00030f  4.491780   4.893673  115.0  -75.0  673767.0  0.327904   28.836567   
seg_0012b5  4.171153   5.922820  152.0 -140.0  625673.0  0.295705   56.217041   
seg_00184e  4.610260   6.946967  248.0 -193.0  691539.0  0.428684  162.112840   
seg_003339  4.531473   4.114133   85.0  -93.0  679721.0  0.061889   41.240413   
seg_0042cc  4.128340   5.797144  177.0 -147.0  619251.0  0.073898   79.537016   
...              ...        ...    ...    ...       ...       ...         ...   
seg_ff4236  4.095773   5.082426   71.0  -50.0  614366.0 -0.001993   12.280456   
seg_ff7478  4.138493   6.031576  164.0 -208.0  620774.0 -0.843307  132.557911   
seg_ff79d9  3.748587   9.467500  310.0 -303.0  562288.0 -0.472015  202.478046   
seg_ffbd6a  4.341687  13.199310  362.0 -319.0  651253.0  0.057967  109.228711   
seg_ffe7cc  4.199720   3.573

False

In [39]:
kfold=KFold(n_splits=5,random_state=2020)

depth_range = [12,16,15,17,18]
minsplit_range = [5,10,20,25,30,3]
minleaf_range = [5,10,15,3]

parameters = dict(max_depth=depth_range,
                  min_samples_split=minsplit_range, 
                  min_samples_leaf=minleaf_range)
rfr = RandomForestRegressor(oob_score=True,random_state=2020)

cv = GridSearchCV(rfr,param_grid=parameters,cv=kfold,scoring='neg_mean_absolute_error',verbose=3)
y_train = y_train['time_to_failure']

F:\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


In [40]:
cv.fit(X_train,y_train)

Fitting 5 folds for each of 120 candidates, totalling 600 fits
[CV] max_depth=12, min_samples_leaf=5, min_samples_split=5 ...........


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  max_depth=12, min_samples_leaf=5, min_samples_split=5, score=-2.246, total=   1.0s
[CV] max_depth=12, min_samples_leaf=5, min_samples_split=5 ...........


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.9s remaining:    0.0s


[CV]  max_depth=12, min_samples_leaf=5, min_samples_split=5, score=-2.421, total=   1.0s
[CV] max_depth=12, min_samples_leaf=5, min_samples_split=5 ...........


...

[CV]  max_depth=17, min_samples_leaf=3, min_samples_split=20, score=-2.419, total=   1.1s
[CV] max_depth=17, min_samples_leaf=3, min_samples_split=20 ..........
[CV]  max_depth=17, min_samples_leaf=3, min_samples_split=20, score=-2.723, total=   1.1s
[CV] max_depth=17, min_samples_leaf=3, min_samples_split=20 ..........
[CV]  max_depth=17, min_samples_leaf=3, min_samples_split=20, score=-1.615, total=   1.1s
[CV] max_depth=17, min_samples_leaf=3, min_samples_split=20 ..........
[CV]  max_depth=17, min_samples_leaf=3, min_samples_split=20, score=-2.654, total=   1.1s
[CV] max_depth=17, min_samples_leaf=3, min_samples_split=25 ..........
[CV]  max_depth=17, min_samples_leaf=3, min_samples_split=25, score=-2.235, total=   1.0s
[CV] max_depth=17, min_samples_leaf=3, min_samples_split=25 ..........
[CV]  max_depth=17, min_samples_leaf=3, min_samples_split=25, score=-2.415, total=   1.0s
[CV] max_depth=17, min_samples_leaf=3, min_samples_split=25 ..........
[CV]  max_depth=17, min_samples_le

[CV]  max_depth=18, min_samples_leaf=10, min_samples_split=5, score=-2.714, total=   0.9s
[CV] max_depth=18, min_samples_leaf=10, min_samples_split=5 ..........
[CV]  max_depth=18, min_samples_leaf=10, min_samples_split=5, score=-1.603, total=   0.9s
[CV] max_depth=18, min_samples_leaf=10, min_samples_split=5 ..........
[CV]  max_depth=18, min_samples_leaf=10, min_samples_split=5, score=-2.629, total=   0.9s
[CV] max_depth=18, min_samples_leaf=10, min_samples_split=10 .........
[CV]  max_depth=18, min_samples_leaf=10, min_samples_split=10, score=-2.253, total=   0.9s
[CV] max_depth=18, min_samples_leaf=10, min_samples_split=10 .........
[CV]  max_depth=18, min_samples_leaf=10, min_samples_split=10, score=-2.407, total=   0.9s
[CV] max_depth=18, min_samples_leaf=10, min_samples_split=10 .........
[CV]  max_depth=18, min_samples_leaf=10, min_samples_split=10, score=-2.714, total=   1.0s
[CV] max_depth=18, min_samples_leaf=10, min_samples_split=10 .........
[CV]  max_depth=18, min_samples

[CV]  max_depth=18, min_samples_leaf=15, min_samples_split=30, score=-1.599, total=   0.8s
[CV] max_depth=18, min_samples_leaf=15, min_samples_split=30 .........
[CV]  max_depth=18, min_samples_leaf=15, min_samples_split=30, score=-2.618, total=   0.8s
[CV] max_depth=18, min_samples_leaf=15, min_samples_split=3 ..........
[CV]  max_depth=18, min_samples_leaf=15, min_samples_split=3, score=-2.244, total=   0.9s
[CV] max_depth=18, min_samples_leaf=15, min_samples_split=3 ..........
[CV]  max_depth=18, min_samples_leaf=15, min_samples_split=3, score=-2.391, total=   0.8s
[CV] max_depth=18, min_samples_leaf=15, min_samples_split=3 ..........
[CV]  max_depth=18, min_samples_leaf=15, min_samples_split=3, score=-2.708, total=   0.9s
[CV] max_depth=18, min_samples_leaf=15, min_samples_split=3 ..........
[CV]  max_depth=18, min_samples_leaf=15, min_samples_split=3, score=-1.599, total=   0.8s
[CV] max_depth=18, min_samples_leaf=15, min_samples_split=3 ..........
[CV]  max_depth=18, min_samples_

[Parallel(n_jobs=1)]: Done 600 out of 600 | elapsed:  9.7min finished


GridSearchCV(cv=KFold(n_splits=5, random_state=2020, shuffle=False),
             error_score=nan,
             estimator=RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                             criterion='mse', max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             max_samples=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators=100, n_jobs=None,
                                             oob_score=True, random_state=2020,
                                             ver

In [41]:
print(cv.best_score_)
print(cv.best_params_)
best=cv.best_estimator_

-2.3117282973989406
{'max_depth': 12, 'min_samples_leaf': 15, 'min_samples_split': 5}


In [42]:
y_pred=best.predict(X_test)
print(y_pred)
type(y_pred)


[ 5.0663359   3.73554437  4.357106   ...  4.42473661  2.06196901
 10.43796878]


numpy.ndarray

In [43]:
y_pred = pd.DataFrame(y_pred)
y_pred.shape
y_pred.head()

,0
0,5.066336
1,3.735544
2,4.357106
3,8.497970
4,6.092020


In [44]:
y_pred.to_csv(r"F:\project\sample1.csv",index=False)